In [76]:
import pandas as pd
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, DefaultDataCollator, create_optimizer
import datasets
import tensorflow as tf

In [2]:
def load_train_val_test_datasets(dataset_path='./../data/datasets/squad'):
    train = pd.read_csv(f'{dataset_path}/train.csv').dropna()
    val = pd.read_csv(f'{dataset_path}/dev.csv').dropna()
    test = pd.read_csv(f'{dataset_path}/test.csv').dropna()
    return train, val, test


def convert_dataframes_to_datasets(dataframes: list):
    return tuple(
        [datasets.Dataset.from_pandas(dataframe, preserve_index=False) for dataframe in
         dataframes])


df_train, df_val, df_test = load_train_val_test_datasets()

train_dataset, val_dataset, test_dataset = convert_dataframes_to_datasets([df_train, df_val, df_test])

In [3]:
train_dataset, val_dataset, test_dataset

(Dataset({
     features: ['index', 'id', 'context', 'question', 'answer_text', 'answer_start'],
     num_rows: 68717
 }),
 Dataset({
     features: ['index', 'id', 'context', 'question', 'answer_text', 'answer_start'],
     num_rows: 14725
 }),
 Dataset({
     features: ['index', 'id', 'context', 'question', 'answer_text', 'answer_start'],
     num_rows: 14726
 }))

In [4]:
model_checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
def tokenize_sample(sample, max_tokens=None, padding=False):
    question = sample['question'].strip()
    context = sample['context'].strip()

    return tokenizer(
        question,
        context,
        max_length=max_tokens,
        padding=padding
    )


tokenized_train_dataset = train_dataset.map(tokenize_sample)
tokenized_val_dataset = val_dataset.map(tokenize_sample)
tokenized_test_dataset = test_dataset.map(tokenize_sample)

print('Max number of tokens in tokenized train dataset: ', len(max(tokenized_train_dataset['input_ids'], key=len)))
print('Max number of tokens in tokenized val dataset: ', len(max(tokenized_val_dataset['input_ids'], key=len)))
print('Max number of tokens in tokenized test dataset: ', len(max(tokenized_test_dataset['input_ids'], key=len)))

Map:   0%|          | 0/68717 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (520 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/14725 [00:00<?, ? examples/s]

Map:   0%|          | 0/14726 [00:00<?, ? examples/s]

Max number of tokens in tokenized train dataset:  870
Max number of tokens in tokenized val dataset:  866
Max number of tokens in tokenized test dataset:  817


In [6]:
max_length = 384


def filter_samples_below_number_of_tokens(dataset, max_tokens: int):
    indices_to_remove = []

    # Find indices of samples where number of tokens exceeds max number of tokens
    for index, sample in enumerate(dataset):
        tokenized_sample = tokenize_sample(sample)
        if len(tokenized_sample['input_ids']) > max_tokens:
            indices_to_remove.append(index)

    # Keep only samples with number of tokens less or equal than max number of tokens
    dataset_indices = range(len(dataset))
    filtered_dataset = dataset.select(
        index for index in dataset_indices if index not in set(indices_to_remove)
    )

    return filtered_dataset


filtered_train_dataset = filter_samples_below_number_of_tokens(train_dataset, max_tokens=max_length)
filtered_val_dataset = filter_samples_below_number_of_tokens(val_dataset, max_tokens=max_length)
filtered_test_dataset = filter_samples_below_number_of_tokens(test_dataset, max_tokens=max_length)

Parameter 'indices'=<generator object filter_samples_below_number_of_tokens.<locals>.<genexpr> at 0x000001890B5E15F0> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [7]:
print('Number of samples in tokenized train dataset before filtering: ', len(train_dataset))
print('Number of samples in tokenized val dataset before filtering: ', len(val_dataset))
print('Number of samples in tokenized test dataset before filtering: ', len(test_dataset))

print('\n---------------\n')

print('Number of samples in tokenized train dataset after filtering: ', len(filtered_train_dataset))
print('Number of samples in tokenized val dataset after filtering: ', len(filtered_val_dataset))
print('Number of samples in tokenized test dataset after filtering: ', len(filtered_test_dataset))

Number of samples in tokenized train dataset before filtering:  68717
Number of samples in tokenized val dataset before filtering:  14725
Number of samples in tokenized test dataset before filtering:  14726

---------------

Number of samples in tokenized train dataset after filtering:  67965
Number of samples in tokenized val dataset after filtering:  14574
Number of samples in tokenized test dataset after filtering:  14553


In [88]:
def tokenize(examples):
    questions = [q.strip() for q in examples['question']]
    contexts = [c.strip() for c in examples['context']]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=max_length,
        padding='max_length'
    )

    return inputs


def preprocess_dataset(dataset):
    dataset = dataset.map(tokenize, batched=True)

    answer_start_indices = dataset['answer_start']
    answer_texts = dataset['answer_text']

    start_positions = []
    end_positions = []

    for index, _ in enumerate(dataset):
        start_char = answer_start_indices[index]
        end_char = start_char + len(answer_texts[index])

        start_positions.append(start_char)
        end_positions.append(end_char)

    dataset = dataset.add_column('start_positions', start_positions)
    dataset = dataset.add_column('end_positions', end_positions)
    return dataset


tokenized_train_dataset = preprocess_dataset(filtered_train_dataset)
tokenized_val_dataset = preprocess_dataset(filtered_val_dataset)
tokenized_test_dataset = preprocess_dataset(filtered_test_dataset)

Map:   0%|          | 0/67965 [00:00<?, ? examples/s]

Map:   0%|          | 0/14574 [00:00<?, ? examples/s]

Map:   0%|          | 0/14553 [00:00<?, ? examples/s]

In [89]:
print(f'All tokenized train dataset entries have {max_length} tokens: ',
      all([len(input_ids) == max_length for input_ids in tokenized_train_dataset['input_ids']]))
print(f'All tokenized val dataset entries have {max_length} tokens: ',
      all([len(input_ids) == max_length for input_ids in tokenized_val_dataset['input_ids']]))
print(f'All tokenized test dataset entries have {max_length} tokens: ',
      all([len(input_ids) == max_length for input_ids in tokenized_test_dataset['input_ids']]))

All tokenized train dataset entries have 384 tokens:  True
All tokenized val dataset entries have 384 tokens:  True
All tokenized test dataset entries have 384 tokens:  True


In [100]:
model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [101]:
data_collator = DefaultDataCollator(return_tensors='tf')

In [102]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_train_dataset,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=4,
)
tf_val_dataset = model.prepare_tf_dataset(
    tokenized_val_dataset,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=4,
)

In [103]:
tf_train_dataset

<PrefetchDataset shapes: ({input_ids: (4, 384), token_type_ids: (4, 384), attention_mask: (4, 384)}, {start_positions: (4,), end_positions: (4,)}), types: ({input_ids: tf.int64, token_type_ids: tf.int64, attention_mask: tf.int64}, {start_positions: tf.int64, end_positions: tf.int64})>

In [ ]:
# <_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(16, 384), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(16, 384), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 384), dtype=tf.int64, name=None)}, {'start_positions': TensorSpec(shape=(16,), dtype=tf.int64, name=None), 'end_positions': TensorSpec(shape=(16,), dtype=tf.int64, name=None)})>

In [86]:
tf_train_dataset = tokenized_train_dataset.to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols=['answer_start', 'answer_end'],
    batch_size=8,
    shuffle=False
)

tf_val_dataset = tokenized_val_dataset.to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols=['answer_start', 'answer_end'],
    batch_size=8,
    shuffle=False
)

In [87]:
tf_train_dataset

<PrefetchDataset shapes: ({input_ids: (None, 384), token_type_ids: (None, 384), attention_mask: (None, 384)}, {answer_start: (None,), answer_end: (None,)}), types: ({input_ids: tf.int64, token_type_ids: tf.int64, attention_mask: tf.int64}, {answer_start: tf.int64, answer_end: tf.int64})>

In [104]:
num_train_epochs = 3
num_train_steps = len(tf_train_dataset) * num_train_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [105]:
model.fit(tf_train_dataset, validation_data=tf_val_dataset, epochs=num_train_epochs)

Epoch 1/3


ResourceExhaustedError:  failed to allocate memory
	 [[node gradient_tape/tf_bert_for_question_answering_2/bert/encoder/layer_._11/intermediate/Gelu/mul_1/Mul (defined at \anaconda3\envs\ml-tf2gpu\lib\site-packages\transformers\modeling_tf_utils.py:1532) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_61406]

Errors may have originated from an input operation.
Input Source operations connected to node gradient_tape/tf_bert_for_question_answering_2/bert/encoder/layer_._11/intermediate/Gelu/mul_1/Mul:
 tf_bert_for_question_answering_2/bert/encoder/layer_._11/intermediate/Gelu/add (defined at \anaconda3\envs\ml-tf2gpu\lib\site-packages\transformers\models\bert\modeling_tf_bert.py:424)

Function call stack:
train_function


In [69]:
model.summary()

Model: "tf_bert_for_question_answering"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108891648 
_________________________________________________________________
qa_outputs (Dense)           multiple                  1538      
Total params: 108,893,186
Trainable params: 108,893,186
Non-trainable params: 0
_________________________________________________________________
